<a href="https://colab.research.google.com/github/geraldoferreira1963/Analise_Sentimento/blob/main/Classificacao_Extrato_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install scikit-learn nltk gensim transformers -q
!pip install requests -q
!pip install scikeras -q
from scikeras.wrappers import KerasClassifier  # Import KerasClassifier from scikeras.wrappers



import requests  # Import the requests library
import tensorflow as tf  # Import tensorflow
import time
import nltk


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
import nltk
import string
from sklearn.naive_bayes import GaussianNB # Import GaussianNB
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import TextVectorization, Embedding, Conv1D, MaxPooling1D, Flatten, Dropout, Dense
from tensorflow.keras.utils import to_categorical # Import to_categorical
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dropout, Dense
from tensorflow.keras.models import Model  # Import Model
from tensorflow.keras.layers import LSTM, Bidirectional # Import LSTM and Bidirectional
from google.colab import files

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('punkt_tab')

# Objetivo : Análise do Dataframe com o dataset carregado, e gerar um primeiro conjunto de estatísticas

In [10]:
def analisar_dataframe(df):
    # Contagem e Impressão do Conteúdo da Classificação
    classificacao_contagem = df['Classificacao'].value_counts()
    print("Distribuição da Classificacao:")

    # Contagem das classes de Classificacao
    data = [['Classificacao', 'Contagem']]
    for classificacao, contagem in classificacao_contagem.items():
        data.append([classificacao, contagem])

    # Formatação dos Dados
    col_width = max(len(str(word)) for row in data for word in row) + 2  # padding
    for row in data:
        print("|" + "|".join(str(word).ljust(col_width) for word in row) + "|")

    # Calcular e Imprimir estatísticas

    df['comprimento_conteudo'] = df['Operacao_Investimento'].apply(len)  # Changed to 'Operacao_Investimento'

    # Imprimir Estatísticas do tamanho do conteúdo
    print("\n\n Estatísticas do Comprimento do Conteúdo:")
    data = [
        ["Menor Comprimento", df['comprimento_conteudo'].min()],
        ["Maior Comprimento", df['comprimento_conteudo'].max()],
        ["Comprimento Mediano", df['comprimento_conteudo'].median()]
    ]

    # Formatar Tabela
    col_width = max(len(str(word)) for row in data for word in row) + 2  # padding
    for row in data:
        print("|" + "|".join(str(word).ljust(col_width) for word in row) + "|")

    comprimento_linha = df['comprimento_conteudo'].max()


    # Cálculo do Vocabulário
    all_words = ' '.join(df['Operacao_Investimento']).lower().split()
    unique_words = set(all_words)
    print("\n\n Número de Palavras:", len(unique_words))

    return(comprimento_linha,len(unique_words))

# Objetivo : Remoção de stop words, pontuações, opcionalmente lematização

In [11]:
def preprocess_text(text, lemmatize=True):
    """Removes stop words, punctuation, and optionally performs lemmatization.

    Args:
        text: The input text string.
        lemmatize: A boolean indicating whether to perform lemmatization.

    Returns:
        The preprocessed text string.
    """

    # Tokenize the text
    tokens = nltk.word_tokenize(text.lower())

    # Remove punctuation
    tokens = [token for token in tokens if token not in string.punctuation]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatize if specified
    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the tokens back into a string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Objetivo : Gerar diversos embeddings, o método será passado como parametro (TF-IDF, Word2Vec, BERT)

In [12]:
def gerar_embedding(X_train, X_test, embedding_type='TF-IDF'):
    if embedding_type == 'TF-IDF':
        vectorizer = TfidfVectorizer(max_features=5000)
        X_train_emb = vectorizer.fit_transform(X_train)
        X_test_emb = vectorizer.transform(X_test)
    elif embedding_type == 'Word2Vec':
        sentences = [text.split() for text in X_train]
        model_w2v = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
        X_train_emb = np.array([np.mean([model_w2v.wv[word] for word in text.split() if word in model_w2v.wv] or [np.zeros(100)], axis=0) for text in X_train])
        X_test_emb = np.array([np.mean([model_w2v.wv[word] for word in text.split() if word in text.split() if word in model_w2v.wv] or [np.zeros(100)], axis=0) for text in X_test])
    elif embedding_type == 'BERT':
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model_bert = BertModel.from_pretrained('bert-base-uncased')

        def get_bert_embedding(text):
            inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
            outputs = model_bert(**inputs)
            return outputs.last_hidden_state[:, 0, :].detach().numpy()

        X_train_emb = np.array([get_bert_embedding(text) for text in X_train])
        X_test_emb = np.array([get_bert_embedding(text) for text in X_test])
    #else:
    #    raise ValueError("Tipo Inválido de Embedding". Choose from 'TF-IDF', 'Word2Vec', or 'BERT')

    return X_train_emb, X_test_emb

# Objetivo : Criar um modelo neural que utilizará como base uma camada CNN (número de filtros, e tamanho são parametros). Adicionalmente existem outros parametros (os quais serão Hiper Parametros) como : dropout e taxa de aprendizado

In [13]:
def criacao_modelo_cnn(num_filters=32, filter_size=3, learning_rate=0.01, dropout_rate=0.2):
    # Camada de Entrada ( entrada já estará vetorizada)
    input_layer = Input(shape=(max_length,))


    # Camada Embedding, pesos seráo aprendidos durante o treinamento
    embedding_layer = Embedding(input_dim=max_tokens, output_dim=128)(input_layer)

    # Camada Convolutional
    cnn_layer = Conv1D(filters=num_filters, kernel_size=filter_size, activation='relu')(embedding_layer)

    # Camada Max pooling
    pooling_layer = MaxPooling1D(pool_size=2)(cnn_layer)

    # Flatten layer
    flatten_layer = Flatten()(pooling_layer)

    # Dropout layer
    dropout_layer = Dropout(dropout_rate)(flatten_layer)

    # Camada de Saída, Softmax, Classificação (Multi-Classe)
    output_layer = Dense(num_classes, activation='softmax')(dropout_layer)


    model = Model(inputs=input_layer, outputs=output_layer)

    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  metrics=['accuracy'])

    return model

# Objetivo : Criar um modelo neural que utilizará como base uma camada LSTM  (número de neurons, bidirecionalidade são parametros). Adicionalmente existem outros parametros (os quais serão Hiper Parametros) como : dropout e taxa de aprendizado

In [14]:
def criacao_modelo_lstm(num_neurons=64, bidirectional=True, learning_rate=0.01, dropout_rate=0.2):

    # Camada de Entrada ( entrada já estará vetorizada)
    input_layer = Input(shape=(max_length,))


    # Camada Embedding, pesos seráo aprendidos durante o treinamento
    embedding_layer = Embedding(input_dim=max_tokens, output_dim=128)(input_layer)

    # LSTM  (bidirectional or unidirectional)
    if bidirectional:
        lstm_layer = Bidirectional(LSTM(num_neurons))(embedding_layer)
    else:
        lstm_layer = LSTM(num_neurons)(embedding_layer)

    # Dropout layer
    dropout_layer = Dropout(dropout_rate)(lstm_layer)

    # Output layer with softmax activation for multi-class classification
    output_layer = Dense(num_classes, activation='softmax')(dropout_layer)

    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  metrics=['accuracy'])

    return model

# Objetivo : Imprimir Resultados dos Treinamentos

In [15]:
def imprimir_resultados_tabela(results):
    """Prints the results in a formatted table."""
    header = ["Model", "Embedding", "Accuracy"]
    col_widths = [max(len(str(x)) for x in col) for col in zip(*([header] + results))]  # Calculate column widths

    # Print header
    print("+" + "+".join("-" * (width + 2) for width in col_widths) + "+")
    print("|" + "|".join(str(x).center(width + 2) for x, width in zip(header, col_widths)) + "|")
    print("+" + "+".join("-" * (width + 2) for width in col_widths) + "+")

    # Print rows
    for row in results:
        print("|" + "|".join(str(x).center(width + 2) for x, width in zip(row, col_widths)) + "|")

    # Print bottom border
    print("+" + "+".join("-" * (width + 2) for width in col_widths) + "+")

# Objetivo : Carga do Dataset

In [16]:
# Corrected URL to download the raw Excel file from GitHub
url = 'https://raw.githubusercontent.com/geraldoferreira1963/Analise_Sentimento/main/carga_inicial_4.xlsx'

response = requests.get(url)

# Save the content to a file with .xlsx extension
with open('carga_inicial_4.xlsx', 'wb') as file:
    file.write(response.content)

# Read the Excel file, explicitly specifying the engine
df = pd.read_excel('carga_inicial_4.xlsx', engine='openpyxl') # Added engine parameter

# Display the first few rows of the DataFrame
df.head()

,Operacao_Investimento,Classificacao
0,Câmbio Padrão : R$5977.29,Operacao_Outras
1,Câmbio Padrão : R$10460.25,Operacao_Outras
2,Câmbio Padrão : R$10460.25,Operacao_Outras
3,Câmbio Padrão : R$9962.14,Operacao_Outras
4,Compra de 60 LTC a $ 43.3,Operacao_Bolsa


Análise Inicial do Dataset

In [17]:
# Verificação Inicial do dataset
max_length, unique_words = analisar_dataframe(df)  # Call the function and assign return values to variables

Distribuição da Classificacao:
|Classificacao    |Contagem         |
|Imposto          |870              |
|Rendimento_Acao  |542              |
|Rendimento_FII   |462              |
|Operacao_Bolsa   |351              |
|Operacao_Outras  |314              |
|Aluguel          |245              |
|Taxa             |209              |
|Resgate_RF       |160              |
|Operacao_RF      |142              |
|Rendimento_RF    |48               |


 Estatísticas do Comprimento do Conteúdo:
|Menor Comprimento    |12                   |
|Maior Comprimento    |80                   |
|Comprimento Mediano  |41.0                 |


 Número de Palavras: 1291


# Modelos Não Neurais - Treinamento & Avaliação

In [ ]:
# Preprocessamento da coluna Operação Investimento remoção de pontuação, stop words
df['Operacao_Investimento'] = df['Operacao_Investimento'].apply(preprocess_text)


# Divisão dos dados
X = df['Operacao_Investimento'].apply(preprocess_text)
y = df['Classificacao']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Geração dos Embeddings
X_train_tfidf, X_test_tfidf = gerar_embedding(X_train, X_test, embedding_type='TF-IDF')
X_train_w2v, X_test_w2v = gerar_embedding(X_train, X_test, embedding_type='Word2Vec')
X_train_bert, X_test_bert = gerar_embedding(X_train, X_test, embedding_type='BERT')


# Modelos e Avaliação
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'SVM': SVC(),
    'Naive Bayes': GaussianNB()
}

# Hiper Parametros
param_grids = {
    'Logistic Regression': {
        'solver': ['liblinear', 'saga'],
        'C': [0.1, 1, 10]
    },
    'SVM': {
        'kernel': ['linear', 'rbf'],
        'C': [0.1, 1, 10]
    }
}

embeddings = {
    'TF-IDF': (X_train_tfidf, X_test_tfidf),
    'Word2Vec': (X_train_w2v, X_test_w2v),
    'BERT': (X_train_bert.squeeze(1), X_test_bert.squeeze(1))
}

results =[]

for model_name, model in models.items():
    for embedding_name, (X_train_emb, X_test_emb) in embeddings.items():
        if model_name in param_grids:
            grid_search = GridSearchCV(model, param_grids[model_name], cv=5, scoring='accuracy')  # cv=5 for 5-fold cross-validation
            grid_search.fit(X_train_emb, y_train)
            model = grid_search.best_estimator_  # Use the best model found
            print(f"Best Hyperparameters: {grid_search.best_params_}")

        # Treinamento do Modelo e Avaliação

        if model_name == 'Naive Bayes' and embedding_name in ['TF-IDF', 'Word2Vec']:
            X_train_emb = X_train_emb.toarray()
            X_test_emb = X_test_emb.toarray()

        model.fit(X_train_emb, y_train)
        y_pred = model.predict(X_test_emb)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy for {model_name} ({embedding_name}): {accuracy:.4f}")
        print(classification_report(y_test, y_pred))  # Print classification report

        # Resultados
        results.append([model_name, embedding_name, accuracy])
    # Impressão dos Resultados
    imprimir_resultados_tabela(results)


KeyboardInterrupt: 

# Modelos Neurais - Pré-Processamento

In [20]:
# Preprocessing
df['Operacao_Investimento'] = df['Operacao_Investimento'].apply(preprocess_text)

# 1. Get all unique values from 'Classificacao' column
all_classes = df['Classificacao'].unique()

# 2. Fit LabelEncoder on all unique values
label_encoder = LabelEncoder()
label_encoder.fit(all_classes)

# Split data
X = df['Operacao_Investimento']
y = df['Classificacao']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Text Vectorization
max_tokens = unique_words
max_length = max_length
vectorizer = TextVectorization(max_tokens=max_tokens, output_sequence_length=max_length)
vectorizer.adapt(X_train)


# Transform your training and test data into numerical representations
X_train_vec = vectorizer(X_train).numpy()  # Vectorize X_train
X_test_vec = vectorizer(X_test).numpy()  # Vectorize X_test

# Transform your training and test labels into numerical representations
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Now you can use to_categorical with the encoded labels
y_train_encoded = to_categorical(y_train_encoded)  # Convert to one-hot encoding
y_test_encoded = to_categorical(y_test_encoded)  # Convert to one-hot encoding


num_classes = y_train_encoded.shape[1]  # Get number of unique classes

# Models and Hyperparameter Tuning

models = {
    'CNN': {
        'create_model': criacao_modelo_cnn,
        'param_grid': {
          #  'model__num_filters': [32, 64],
         #   'model__filter_size': [3, 5],
            'model__learning_rate': [0.01, 0.001],
            'model__dropout_rate': [0.2, 0.5],
            'batch_size': [32, 64],
            'epochs': [10, 20]
        }
    },
    'LSTM': {
        'create_model': criacao_modelo_lstm,
        'param_grid': {
            'model__num_neurons': [32, 64],
           # 'model__bidirectional': [True, False],
            'model__learning_rate': [0.01, 0.001],
            'model__dropout_rate': [0.2, 0.5],
            'batch_size': [32, 64],
            'epochs': [10, 20]
        }
    }
}

# Store results for the table
results = []

for model_name, model_info in models.items():
    print(f"Training and evaluating {model_name} model...")

    # Create and tune the model using KerasClassifier and GridSearchCV
    model = KerasClassifier(model=model_info['create_model'], verbose=0)
    grid_search = GridSearchCV(estimator=model, param_grid=model_info['param_grid'], cv=5, scoring='accuracy',verbose=2)
    grid_result = grid_search.fit(X_train_vec, y_train_encoded)

    # Print best hyperparameters
    print(f"Best Hyperparameters for {model_name}: {grid_result.best_params_}")

    # Evaluate the best model
    best_model = grid_result.best_estimator_  # Get the best KerasClassifier instance
    trained_keras_model = best_model.model_  # Access the trained Keras model

    _, accuracy = trained_keras_model.evaluate(X_test_vec, y_test_encoded, verbose=1)  # changed here
    print(f"Accuracy for {model_name}: {accuracy:.4f}")
    # --- Add this section to calculate y_pred ---
    y_pred_probs = trained_keras_model.predict(X_test_vec)  # Get predicted probabilities  # changed here
    y_pred = np.argmax(y_pred_probs, axis=1)      # Get predicted class indices
    y_pred = label_encoder.inverse_transform(y_pred) # Convert indices back to original labels
    print(classification_report(y_test, y_pred))

    # Store results
    results.append([model_name, accuracy])

# Print the results table
imprimir_resultados_tabela(results)

NameError: name 'to_categorical' is not defined

# Modelos Neurais - Preparação dos Dados

In [32]:
df['Operacao_Investimento'] = df['Operacao_Investimento'].apply(preprocess_text)

#
all_classes = df['Classificacao'].unique()

# Gerando Encoders
label_encoder = LabelEncoder()
label_encoder.fit(all_classes)

# Divisão dos Dados
X = df['Operacao_Investimento']
y = df['Classificacao']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vetorização da Operação Investimento
max_tokens = unique_words
max_length = max_length
vectorizer = TextVectorization(max_tokens=max_tokens, output_sequence_length=max_length)
vectorizer.adapt(X_train)


# Transformando os dados de treinamento e teste em vetores
X_train_vec = vectorizer(X_train).numpy()
X_test_vec = vectorizer(X_test).numpy()

# Transformandos os dados de Label (Classificação ) em representações númericas
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Realizando o one-hot encoding de Classificação
y_train_encoded = to_categorical(y_train_encoded)
y_test_encoded = to_categorical(y_test_encoded)


num_classes = y_train_encoded.shape[1]

# Models e  Hiper Parametros

models = {
    'CNN': {
        'create_model': criacao_modelo_cnn,
        'param_grid': {
          #  'model__num_filters': [32, 64], # comentado decorrente da falta de memória para considerar esta variável no espaço de busca
         #   'model__filter_size': [3, 5],   # comentado decorrente da falta de memória para considerar esta variável no espaço de busca
            'model__learning_rate': [0.01, 0.001],
            'model__dropout_rate': [0.2, 0.5],
            'batch_size': [32, 64],
            'epochs': [10, 20]
        }
    },
    'LSTM': {
        'create_model': criacao_modelo_lstm,
        'param_grid': {
            'model__num_neurons': [32, 64],
           # 'model__bidirectional': [True, False],
            'model__learning_rate': [0.01, 0.001],
            'model__dropout_rate': [0.2, 0.5],
            'batch_size': [32, 64],
            'epochs': [10, 20]
        }
    }
}

# Modelos Neurais - Treinamento dos Modelos Neurais e Geração dos Arquivos

In [43]:
print(best_models)

{'CNN': KerasClassifier(
	model=<function criacao_modelo_cnn at 0x79c3ca3eef80>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=64
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	class_weight=None
	model__dropout_rate=0.2
	model__learning_rate=0.01
), 'LSTM': KerasClassifier(
	model=<function criacao_modelo_lstm at 0x79c3ca3ef640>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=32
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=20
	class_weight=None
	model__dropout_rate=0.5
	model__learning_rate=0.001
	model__num_neurons=32
)}


In [41]:
best_models = {}

for model_name, model_info in models.items():
    print(f"Treinamento do modelo  {model_name} ...")
    best_accuracy = 0.0  # Inicializando

    # Iniciando a busca os melhores hiper parametros
    model = KerasClassifier(model=model_info['create_model'], verbose=1)
    grid_search = GridSearchCV(estimator=model, param_grid=model_info['param_grid'], cv=5, scoring='accuracy', verbose=2)
    grid_result = grid_search.fit(X_train_vec, y_train_encoded)

    # Imprimeir os melhores hiper parametros
    print(f"Melhores Hiper parametros para o  {model_name}: {grid_result.best_params_}")

    # Update best_accuracy and best_model if current model is better
    current_accuracy = grid_result.best_score_
    if current_accuracy > best_accuracy:
        best_accuracy = current_accuracy
        best_models[model_name] = grid_result.best_estimator_  # Store the best KerasClassifier

# 2. Saving the Best Models
for model_name, best_model in best_models.items():
    filename = f"{model_name}_best_model.keras"
    best_model.model_.save(filename)  # Salvando o modelo
    print(f"Saved best {model_name} model to {filename}")
    files.download(filename)  # Optionally download the model file

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Modelos Neurais - Avaliação dos Modelos

In [42]:
results = []  # Lista para armazenar os resultados

for model_name, best_model in best_models.items():
    filename = f"{model_name}_best_model.keras"

    # Load the trained Keras model
    trained_keras_model = tf.keras.models.load_model(filename)

    # Evaluate the model
    _, accuracy = trained_keras_model.evaluate(X_test_vec, y_test_encoded, verbose=1)
    print(f"Acurácia para o  {model_name}: {accuracy:.4f}")

    # Realizando a Predição
    y_pred_probs = trained_keras_model.predict(X_test_vec)  # Get predicted probabilities  # changed here
    y_pred = np.argmax(y_pred_probs, axis=1)      # Get predicted class indices
    y_pred = label_encoder.inverse_transform(y_pred) # Convert indices back to original labels

    # Gerar e Imprimir a Classificação do Report
    print(classification_report(y_test, y_pred))

    # Atualizando resultados
    results.append([model_name, accuracy])

#Impressão e Formatação dos Resultados
imprimir_resultados_tabela(results)

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.9894 - loss: 0.0644
Acurácia para o  CNN: 0.9895
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
                 precision    recall  f1-score   support

        Aluguel       1.00      1.00      1.00        53
        Imposto       1.00      1.00      1.00       167
 Operacao_Bolsa       0.99      1.00      0.99        71
Operacao_Outras       1.00      0.97      0.98        61
    Operacao_RF       1.00      0.96      0.98        25
Rendimento_Acao       0.97      0.99      0.98       115
 Rendimento_FII       0.98      0.99      0.98        86
  Rendimento_RF       1.00      0.80      0.89        10
     Resgate_RF       1.00      1.00      1.00        31
           Taxa       0.98      1.00      0.99        50

       accuracy                           0.99       669
      macro avg       0.99      0.97      0.98       669
   weighted avg       0.99      0.99      0.99       669

21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.990